In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
import re

In [2]:
# procedure table
proce = pd.read_excel('IndividualTables.xlsx', index_col=None, sheet_name='procedures')
proce = proce.drop(columns = ['ProcedurePrimaryKey'])
p = pd.get_dummies(proce['ProcedurePerformed'], prefix='procedure')
procecols = list(p.columns)
u = list(p.columns)
u.append('isALS_LOC')
proce_df = pd.concat([proce.drop(columns = ['ProcedurePerformed']), p], axis = 1)
proce_df = proce_df.groupby(['PatientID'])[u].apply(lambda x : x.sum()).reset_index()

In [3]:
# medical table
med = pd.read_excel('IndividualTables.xlsx', index_col=None, sheet_name='medications').dropna()
med = med.drop(columns = ['MedicationPrimaryKey'])
m = pd.get_dummies(med['MedicationGiven'], prefix='medication')
medcols = list(m.columns)
u = list(m.columns)
u.append('isALS_LOC')
med_df = pd.concat([med.drop(columns = ['MedicationGiven']), m], axis = 1)
med_df = med_df.groupby(['PatientID'])[u].apply(lambda x : x.sum()).reset_index()

In [4]:
# patient table
pati = pd.read_excel('IndividualTables.xlsx', index_col=None, sheet_name='patients')
pati = pati.drop(columns = ['TransportHospitalCode', 'PatientDisposition'])
pati = pati[~pati.PatientID.isin([918485, 997953, 918485])]

In [5]:
# reason, impression, age, hosp, transloc
reason = pati['ReasonForChoosingHospital']
imp = pati[['PrimaryImpression', 'SecondaryImpression']]
age = pati['PatientAge']
hosp = pati['TransportHospitalName']
transloc =pati['TransportLOC']

In [6]:
# impression
imp1 = pd.get_dummies(imp['PrimaryImpression'], prefix='impression')
imp2 = pd.DataFrame(imp['SecondaryImpression'])
def clean_comma(col):
    return col.str.replace(' (ETOH)', '',regex = False).str.replace('Sedative, Hypnotic', 'Sedative Hypnotic', regex = False).str.replace(', With', 'With',regex = False).str.replace(', unspecified','unspecified',regex = False).str.replace(', suspected','suspected',regex = False).str.replace('Wrist, Hand,','Wrist Hand',regex = False).str.replace('Chemicals, Gases, Fumes,','Chemicals Gases Fumes',regex = False)
imp2['SecondaryImpression'] = imp['SecondaryImpression'].astype(str).str.replace('"', "").str.strip()
imp2 = clean_comma(imp2['SecondaryImpression'].astype(str))
z = imp2.str.split(',', expand=True)
r = pd.get_dummies(z, prefix='impression')
r.columns = r.columns.str.replace('_ ', '_', regex = False)
cols = set(r.columns)
result = pd.DataFrame()
for col in cols:
    try:
        r[col].sum(axis = 1)
        result[col] = r[col].sum(axis = 1)
    except:
        result[col] = r[col]
imp = pd.concat([imp1, result], axis = 1)
impcols = list(set(imp.columns))
imp_df = pd.DataFrame()
for col in impcols:
    try:
        imp[col].sum(axis = 1)
        imp_df[col] = imp[col].sum(axis = 1)
    except:
        imp_df[col] = imp[col]

In [7]:
# reason
reason = pati['ReasonForChoosingHospital'].fillna('No Hospital')
def clean_reason(col):
    return col.str.replace('Trauma, STEMI, Stroke', 'Trauma STEMI Stroke', regex = False)
reason = clean_reason(reason)
reason = reason.str.split(',', expand=True)
reason = pd.get_dummies(reason,prefix='reason')
result = pd.DataFrame()
cols = list(set(reason.columns))
for col in cols:
    try:
        reason[col].sum(axis = 1)
        result[col] = reason[col].sum(axis = 1)
    except:
        result[col] = reason[col]
reason_df = pd.DataFrame(result)
reasoncols = list(reason_df.columns)

In [8]:
# patient age
age = pati['PatientAge']
age_df = pd.get_dummies(age, prefix='patient_age')
agecols = list(age_df.columns)

In [9]:
# TransportLOC
transloc = pati['TransportLOC'].fillna('No Transport LOC')
transloc_df = pd.get_dummies(transloc, prefix='transloc')
transloccols = list(transloc_df.columns)

In [10]:
# TransportHospitalName
hosp = pati['TransportHospitalName'].fillna('No Hospital')
hosp_df = pd.get_dummies(hosp, prefix='hospital_name')
hospcols = list(hosp_df.columns)

In [11]:
# combine
pati_base = pati[['PrimaryKey', 'PatientID']]
new_pati = pd.concat([pati_base, transloc_df, hosp_df, reason_df, age_df, imp_df], axis = 1)
full_pati = new_pati.merge(proce_df, on = 'PatientID',how = 'left').fillna(0)
full_pati = full_pati.merge(med_df, on = 'PatientID',how = 'left').fillna(0)
full_pati['isALS_LOC'] = full_pati['isALS_LOC_x'] + full_pati['isALS_LOC_y']
full_pati = full_pati.drop(columns = ['isALS_LOC_x', 'isALS_LOC_y'])
fianl_pati = full_pati.groupby('PrimaryKey')[list(full_pati.columns[2:])].apply(lambda x : x.sum()).reset_index()
#fianl_pati

In [12]:
# unit table
unit = pd.read_excel('IndividualTables.xlsx', index_col=None, sheet_name='units').dropna(subset = ['UnitStation'])

In [13]:
# action
def clean_comma_act(col):
    return col.str.replace(', ', ' ')
act1 = clean_comma_act(unit['UnitPrimaryActionTaken']).fillna('No Action')
act2 = clean_comma_act(unit['UnitOtherActionsTaken']).fillna('missing')
a1 = act1.str.split(',', expand=True)
a1 = pd.get_dummies(a1, prefix='unit_action')
a2 = act2.str.split(',', expand=True)
a2 = pd.get_dummies(a2, prefix='unit_action')
cols = set(a2.columns)
result = pd.DataFrame()
for col in cols:
    try:
        a2[col].sum(axis = 1)
        result[col] = a2[col].sum(axis = 1)
    except:
        result[col] = a2[col]
a = pd.concat([a1, result], axis = 1)
actcols = list(set(a.columns))
act_df = pd.DataFrame()
for col in actcols:
    try:
        a[col].sum(axis = 1)
        act_df[col] = a[col].sum(axis = 1)
    except:
        act_df[col] = a[col]
act_df = act_df.drop(columns = 'unit_action_missing')

In [14]:
# type
unittype = unit['UnitType']
unittype_df = pd.get_dummies(unittype, prefix='unit_type')
unittypecols = list(unittype_df.columns)

In [15]:
# unit station
unitsta = unit['UnitStation']
unitsta = unitsta[~unitsta.isin([3, 33, 'PRV', 'TA'])]
unitsta_df = pd.get_dummies(unitsta, prefix='unit_station')
unitstacols = list(unitsta_df.columns)

In [16]:
unit_base = unit[['PrimaryKey']]
new_unit = pd.concat([unit_base, act_df, unitsta_df, unittype_df],axis = 1)
final_unit = new_unit.groupby('PrimaryKey')[list(new_unit.columns[1:])].apply(lambda x : x.sum()).reset_index()
#final_unit

In [17]:
inci = pd.read_excel('IndividualTables.xlsx', index_col=None, sheet_name='incidents')
inci = inci.drop(columns = ['ShiftDay', 'TourOfShift', 'FinalIncidentTypeDescription'])

In [18]:
inci['IncidentFirstDue'] = inci['IncidentFirstDue'].fillna('Outside Fairfax')
def clean_ini(row):
    if row['InitialIncidentType'] == 'MEDICAL':
        return 'BLS'
    else:
        return row['InitialIncidentType']
def clean_dis(row):
    if row['DispatchedIncidentType'] == 'MEDICAL':
        return 'BLS'
    else:
        return row['DispatchedIncidentType']
def clean_arri(row):
    if row['ArrivedIncidentType'] == 'MEDICAL':
        return 'BLS'
    else:
        return row['ArrivedIncidentType']
def clean_fina(row):
    if row['FinalIncidentType'] == 'MEDICAL':
        return 'BLS'
    else:
        return row['FinalIncidentType']
inci['InitialIncidentType'] = inci.apply(lambda row: clean_ini(row), axis = 1)
inci['DispatchedIncidentType'] = inci.apply(lambda row: clean_dis(row), axis = 1)
inci['ArrivedIncidentType'] = inci.apply(lambda row: clean_arri(row), axis = 1)
inci['FinalIncidentType'] = inci.apply(lambda row: clean_fina(row), axis = 1)

In [19]:
inci['CallDTMonth'] = inci['CallConfirmedDT'].dt.month
inci['CallDTHour'] = inci['CallConfirmedDT'].dt.hour
inci['CallDTDayofWeek'] = inci['CallConfirmedDT'].dt.dayofweek

In [20]:
# initial
ini = inci['InitialIncidentType']
i = pd.get_dummies(ini, prefix='InitialIncidentType')
# dis
dis = inci['DispatchedIncidentType']
d = pd.get_dummies(dis, prefix='DispatchedIncidentType')
# arri
arri = inci['ArrivedIncidentType']
a = pd.get_dummies(arri, prefix='ArrivedIncidentType')
# final
fina = inci['FinalIncidentType']
f = pd.get_dummies(fina, prefix='FinalIncidentType')
# fdue
fdue = inci['IncidentFirstDue'].apply(str)
fdue = pd.get_dummies(fdue, prefix='IncidentFirstDue')
inci = pd.concat([inci, fdue, i, d, a, f],axis = 1)
inci = inci.drop(columns = ['IncidentFirstDue', 'InitialIncidentType', 'DispatchedIncidentType', 'ArrivedIncidentType', 'FinalIncidentType', 'CallConfirmedDT'])

In [21]:
final_df = inci.merge(final_unit, on = 'PrimaryKey', how='inner')
final_df = final_df.merge(fianl_pati, on = 'PrimaryKey', how='inner')

In [22]:
final_df = final_df.drop_duplicates(subset = 'PrimaryKey')

In [23]:
def clean_alsloc(row):
    if row['isALS_LOC'] >= 1.0:
        return 1
    else:
        return 0
final_df['isALS_LOC'] = final_df.apply(lambda row: clean_alsloc(row), axis = 1)

In [24]:
# Write CSV
#final_df.to_csv('MasterData_Restructured4.txt', sep = '|')